## 예술의전당 콘서트홀 가격 모델

GOAL : 예술의전당 콘서트홀의 좌석별 위치적 특성을 나타내는 파생변수를 생성

0. 자료 조사 : '예술의전당 콘서트홀 좌석배치도 (2023, 2018)', '예술의전당 콘서트홀 도면 (나라장터 2023)' 
1. 데이터 라벨링 : AutoCAD, excel을 이용해서 좌석별 (x, y, z) 좌표 라벨링
2. 데이터 라벨링 결과물 확인 : describe, scatter plot 시각화
3. 좌표 단위 수정 : mm => cm, round
4. 파생변수 생성 : '거리', '시야각' 등의 최종적으로 필요한 변수

In [7]:
import pandas as pd
import numpy as np
from glob import glob

In [8]:
# 좌석좌표 엑셀 파일 로드
print(glob('*.xlsx'))
df = pd.read_excel(glob('*.xlsx')[0])

# shape, 데이터 확인
print(df.shape)
df.head()

['예술의전당 콘서트홀 좌석좌표.xlsx']
(2505, 8)


,층,블록,열,넘버,전체_좌석,X,Y,Z
0,1층,A블록,1,1,1층 A블록1열 1,14512.2087,5422.5042,60874
1,1층,A블록,1,2,1층 A블록1열 2,14059.5465,5548.9833,60874
2,1층,A블록,1,3,1층 A블록1열 3,13606.8844,5675.4624,60874
3,1층,A블록,1,4,1층 A블록1열 4,13154.2222,5801.9416,60874
4,1층,A블록,1,5,1층 A블록1열 5,12695.7535,5906.1735,60874


In [9]:
# 무대 중앙을 원점(0, 0, 0)으로 설정
# 엑셀파일의 좌표는 건축도면을 기준으로 되어 있음. > Z좌표에서 무대의 바닥 높이 만큼 빼준다.
df['Z'] = (df['Z'] - 61800)

# X, Y, Z 좌표는 기존에 float으로 되어 있음 (mm 기준) => int (cm 기준)으로 변경
df['X'] = (round(df['X'], -1) / 10).astype(int)
df['Y'] = (round(df['Y'], -1) / 10).astype(int)
df['Z'] = (round(df['Z'], -1) / 10).astype(int)

df.head()

,층,블록,열,넘버,전체_좌석,X,Y,Z
0,1층,A블록,1,1,1층 A블록1열 1,1451,542,-93
1,1층,A블록,1,2,1층 A블록1열 2,1406,555,-93
2,1층,A블록,1,3,1층 A블록1열 3,1361,568,-93
3,1층,A블록,1,4,1층 A블록1열 4,1315,580,-93
4,1층,A블록,1,5,1층 A블록1열 5,1270,591,-93


In [5]:
# 1. 무대-좌석 간의 거리 파생변수 생성
df['거리'] = round(np.sqrt(df['X']**2 + df['Y']**2 + df['Z']**2)).astype(int)

# 2. 시야각 파생변수 생성
df['좌우시야각'] = (round(np.rad2deg(np.arctan(df['X']/ df['Y'])), 1))
df['상하시야각'] = (round(np.rad2deg(np.arcsinh(df['Z'] / df['거리'])), 1))

df.head()

,층,블록,열,넘버,전체_좌석,X,Y,Z,거리,좌우시야각,상하시야각
0,1층,A블록,1,1,1층 A블록1열 1,1451,542,-93,1552,69.5,-3.4
1,1층,A블록,1,2,1층 A블록1열 2,1406,555,-93,1514,68.5,-3.5
2,1층,A블록,1,3,1층 A블록1열 3,1361,568,-93,1478,67.3,-3.6
3,1층,A블록,1,4,1층 A블록1열 4,1315,580,-93,1440,66.2,-3.7
4,1층,A블록,1,5,1층 A블록1열 5,1270,591,-93,1404,65.0,-3.8


In [6]:
df.loc[df['X'] == (df.loc[df['층'] == '2층']['X'].min())]

,층,블록,열,넘버,전체_좌석,X,Y,Z,거리,좌우시야각,상하시야각
2019,2층,D블록,8,4,2층 D블록8열 4,-4086,4982,923,6509,-39.4,8.1


In [178]:
df[df['블록'] == 'H블록'].head(20)

,층,블록,열,넘버,전체_좌석,X,Y,Z,거리,좌우시야각,상하시야각
1278,합창석,H블록,1,1,합창석 H블록1열 1,820,-900,300,1254,-42.3,13.6
1279,합창석,H블록,1,2,합창석 H블록1열 2,836,-851,300,1230,-44.5,13.8
1280,합창석,H블록,1,3,합창석 H블록1열 3,852,-802,300,1208,-46.7,14.1
1281,합창석,H블록,1,4,합창석 H블록1열 4,868,-753,300,1188,-49.1,14.3
1282,합창석,H블록,1,5,합창석 H블록1열 5,884,-704,300,1169,-51.5,14.5
1283,합창석,H블록,1,6,합창석 H블록1열 6,900,-655,300,1153,-54.0,14.7
1284,합창석,H블록,1,7,합창석 H블록1열 7,916,-606,300,1139,-56.5,14.9
1285,합창석,H블록,1,8,합창석 H블록1열 8,932,-557,300,1126,-59.1,15.1
1286,합창석,H블록,1,9,합창석 H블록1열 9,948,-508,300,1117,-61.8,15.2
1287,합창석,H블록,1,10,합창석 H블록1열 10,964,-459,300,1109,-64.5,15.3


In [177]:
df[df['블록'] == 'G블록'].head(20)

,층,블록,열,넘버,전체_좌석,X,Y,Z,거리,좌우시야각,상하시야각
1250,합창석,G블록,1,1,합창석 G블록1열 1,-689,-995,300,1247,34.7,13.7
1251,합창석,G블록,1,2,합창석 G블록1열 2,-638,-995,300,1219,32.7,14.0
1252,합창석,G블록,1,3,합창석 G블록1열 3,-587,-995,300,1194,30.5,14.2
1253,합창석,G블록,1,4,합창석 G블록1열 4,-536,-995,300,1169,28.3,14.5
1254,합창석,G블록,1,5,합창석 G블록1열 5,-485,-995,300,1147,26.0,14.8
1255,합창석,G블록,1,6,합창석 G블록1열 6,-434,-995,300,1126,23.6,15.1
1256,합창석,G블록,1,7,합창석 G블록1열 7,-383,-995,300,1108,21.1,15.3
1257,합창석,G블록,1,8,합창석 G블록1열 8,-332,-995,300,1091,18.5,15.6
1258,합창석,G블록,1,9,합창석 G블록1열 9,-281,-995,300,1077,15.8,15.8
1259,합창석,G블록,1,10,합창석 G블록1열 10,-230,-995,300,1064,13.0,15.9
